In [1]:
import pandas as pd
import json
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
df = pd.read_json("spaceX_Lunch.json")

df.tail(5)

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
200,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,NaN,[],None,[],[],[],[],5e9e4501f509094ba4566f84,198,Transporter-6,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T19:00:00-05:00,month,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,None,633f72580531f07b4fdf59c6
201,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,NaN,[],None,[],[],[],[63161760ffc78f3b8567071c],5e9e4502f509092b78566f87,199,TTL-1,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T16:00:00-08:00,month,True,"[{'core': None, 'flight': None, 'gridfins': No...",True,False,None,63161384ffc78f3b8567070e
202,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,NaN,[],None,[],[],[],[6243b896af52800c6e91926e],5e9e4501f509094ba4566f84,200,WorldView Legion 1 & 2,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T19:00:00-05:00,month,True,"[{'core': None, 'flight': None, 'gridfins': No...",True,False,None,6243ae58af52800c6e91925a
203,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69974db09d1ed,NaN,[],None,[],[],[],[],5e9e4502f509094188566f88,202,Viasat-3 & Arcturus,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T19:00:00-05:00,month,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,None,633f72130531f07b4fdf59c3
204,"{'reused': None, 'recovery_attempt': None, 're...","{'patch': {'small': None, 'large': None}, 'red...",None,NaN,False,NaN,5e9d0d95eda69973a809d1ec,NaN,[],None,[],[],[],[6243b788af52800c6e91926b],5e9e4501f509094ba4566f84,203,O3b mPower 3.4,2022-12-01T00:00:00.000Z,1669852800,2022-11-30T19:00:00-05:00,month,True,"[{'core': None, 'flight': None, 'gridfins': No...",True,False,None,6243ae7daf52800c6e91925b


# El objetivo

1. Haz una exploración de los datos para identificar columnas que deben ser limpiadas y/o transformadas. Incluye en la limpieza de datos el manejo de valores faltantes y duplicados.

#### Exploración

In [3]:
print(f"El número de filas que tenemos es {df.shape[0]}, y el número de columnas es {df.shape[1]}")

El número de filas que tenemos es 205, y el número de columnas es 27


In [4]:
df.columns

Index(['fairings', 'links', 'static_fire_date_utc', 'static_fire_date_unix',
       'net', 'window', 'rocket', 'success', 'failures', 'details', 'crew',
       'ships', 'capsules', 'payloads', 'launchpad', 'flight_number', 'name',
       'date_utc', 'date_unix', 'date_local', 'date_precision', 'upcoming',
       'cores', 'auto_update', 'tbd', 'launch_library_id', 'id'],
      dtype='object')

In [5]:
df.describe(include = "object").T

,count,unique,top,freq
fairings,169,38,"{'reused': None, 'recovery_attempt': None, 're...",61
links,205,191,"{'patch': {'small': None, 'large': None}, 'red...",15
static_fire_date_utc,121,121,2006-03-17T00:00:00.000Z,1
rocket,205,3,5e9d0d95eda69973a809d1ec,195
failures,205,6,[],200
details,134,133,This mission launches the eighteenth batch of ...,2
crew,205,9,[],197
ships,205,90,[],85
capsules,205,22,[],170
payloads,205,192,[],12


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
static_fire_date_unix,121.0,1.520206e+09,9.080036e+07,1.142554e+09,1.475417e+09,1.529789e+09,1.589368e+09,1.650464e+09
window,117.0,2.568974e+03,4.389948e+03,0.000000e+00,0.000000e+00,0.000000e+00,5.280000e+03,1.830000e+04
success,186.0,9.731183e-01,1.621743e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
flight_number,205.0,1.028146e+02,5.902911e+01,1.000000e+00,5.200000e+01,1.030000e+02,1.540000e+02,2.030000e+02
date_unix,205.0,1.566077e+09,1.037329e+08,1.143239e+09,1.513992e+09,1.601984e+09,1.647660e+09,1.670198e+09


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fairings               169 non-null    object 
 1   links                  205 non-null    object 
 2   static_fire_date_utc   121 non-null    object 
 3   static_fire_date_unix  121 non-null    float64
 4   net                    205 non-null    bool   
 5   window                 117 non-null    float64
 6   rocket                 205 non-null    object 
 7   success                186 non-null    float64
 8   failures               205 non-null    object 
 9   details                134 non-null    object 
 10  crew                   205 non-null    object 
 11  ships                  205 non-null    object 
 12  capsules               205 non-null    object 
 13  payloads               205 non-null    object 
 14  launchpad              205 non-null    object 
 15  flight

In [8]:
df_cat = df.select_dtypes(include = "O")
df_cat.head()

for col in df_cat.columns:
    print("Los valores unicos para la columna: ", col.upper())
    display(df[col].value_counts().reset_index().head())

Los valores unicos para la columna:  FAIRINGS


,fairings,count
0,"{'reused': None, 'recovery_attempt': None, 're...",61
1,"{'reused': False, 'recovery_attempt': False, '...",49
2,"{'reused': False, 'recovery_attempt': True, 'r...",9
3,"{'reused': False, 'recovery_attempt': True, 'r...",3
4,"{'reused': True, 'recovery_attempt': True, 're...",3


Los valores unicos para la columna:  LINKS


,links,count
0,"{'patch': {'small': None, 'large': None}, 'red...",15
1,{'patch': {'small': 'https://images2.imgbox.co...,1
2,{'patch': {'small': 'https://images2.imgbox.co...,1
3,{'patch': {'small': 'https://images2.imgbox.co...,1
4,{'patch': {'small': 'https://images2.imgbox.co...,1


Los valores unicos para la columna:  STATIC_FIRE_DATE_UTC


...

,cores,count
0,"[{'core': None, 'flight': None, 'gridfins': Tr...",10
1,"[{'core': None, 'flight': None, 'gridfins': No...",6
2,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",1
3,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",1
4,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",1


Los valores unicos para la columna:  LAUNCH_LIBRARY_ID


,launch_library_id,count
0,f31702e8-6353-4c9a-932c-5bd104717500,1
1,985f1cc1-82c1-4a89-b2cc-e9dc91829a0e,1
2,dfd4f0e0-0ab4-494d-bd88-1b93b934b269,1
3,134eb787-244e-4131-8b03-c9fbd0a11efc,1
4,896d876d-e834-4810-8a5e-44d6b6a42630,1


Los valores unicos para la columna:  ID


,id,count
0,5eb87cd9ffd86e000604b32a,1
1,5eb87cdaffd86e000604b32b,1
2,5eb87cdbffd86e000604b32c,1
3,5eb87cdbffd86e000604b32d,1
4,5eb87cdcffd86e000604b32e,1


In [9]:
# Identificar los números duplicados
duplicados = df[df['id'].duplicated()]

# Extraer solo la columna 'employeenumber' de los duplicados
numeros_duplicados = duplicados['id']

# Imprimir los números duplicados
print(numeros_duplicados)

# si solo quieres los valores unicos de los duplicados.
print(numeros_duplicados.unique())

# si quieres contar los duplicados.
print(numeros_duplicados.value_counts())

Series([], Name: id, dtype: object)
[]
Series([], Name: count, dtype: int64)


In [10]:
porc_nulos = (df.isnull().sum() / df.shape[0]) * 100
porc_nulos

fairings                 17.560976
links                     0.000000
static_fire_date_utc     40.975610
static_fire_date_unix    40.975610
net                       0.000000
window                   42.926829
rocket                    0.000000
success                   9.268293
failures                  0.000000
details                  34.634146
crew                      0.000000
ships                     0.000000
capsules                  0.000000
payloads                  0.000000
launchpad                 0.000000
flight_number             0.000000
name                      0.000000
date_utc                  0.000000
date_unix                 0.000000
date_local                0.000000
date_precision            0.000000
upcoming                  0.000000
cores                     0.000000
auto_update               0.000000
tbd                       0.000000
launch_library_id        64.878049
id                        0.000000
dtype: float64

In [11]:
# lo convertimos a DataFrame
df_nulos = pd.DataFrame(porc_nulos, columns = ["%_nulos"])

# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos[df_nulos["%_nulos"] > 0]

,%_nulos
fairings,17.560976
static_fire_date_utc,40.975610
static_fire_date_unix,40.975610
window,42.926829
success,9.268293
details,34.634146
launch_library_id,64.878049


#### Limpieza

In [12]:
df.head(3)

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b
2,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",5e9e4502f5090995de566f86,3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",True,False,None,5eb87cdbffd86e000604b32c


In [13]:
#buscar una realción 'static_fire_date_utc' y 'static_fire_date_unix'
columnas_seleccionadas = df[['static_fire_date_utc', 'static_fire_date_unix']]
columnas_seleccionadas

,static_fire_date_utc,static_fire_date_unix
0,2006-03-17T00:00:00.000Z,1.142554e+09
1,None,NaN
2,None,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09
4,None,NaN
...,...,...
200,None,NaN
201,None,NaN
202,None,NaN
203,None,NaN


In [14]:
df.drop("static_fire_date_unix", axis = 1, inplace = True)
df.head(1)

,fairings,links,static_fire_date_utc,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a


In [15]:
#compar 'flight_number'y 'date_unix' para visualizar
columnas_seleccionadas = df[['flight_number', 'date_unix']]
columnas_seleccionadas

,flight_number,date_unix
0,1,1143239400
1,2,1174439400
2,3,1217734440
3,4,1222643700
4,5,1247456100
...,...,...
200,198,1669852800
201,199,1669852800
202,200,1669852800
203,202,1669852800


In [16]:
df["flight_number"].duplicated().sum()

np.int64(4)

In [17]:
df.drop("flight_number", axis = 1, inplace = True)

df.head(1)

,fairings,links,static_fire_date_utc,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a


In [18]:
#ver columna, diccionarios
df["fairings"].value_counts()

fairings
{'reused': None, 'recovery_attempt': None, 'recovered': None, 'ships': []}                                                                                      61
{'reused': False, 'recovery_attempt': False, 'recovered': False, 'ships': []}                                                                                   49
{'reused': False, 'recovery_attempt': True, 'recovered': False, 'ships': ['5ea6ed2e080df4000697c908']}                                                           9
{'reused': False, 'recovery_attempt': True, 'recovered': True, 'ships': ['5ea6ed2e080df4000697c908']}                                                            3
{'reused': True, 'recovery_attempt': True, 'recovered': None, 'ships': ['5ea6ed2e080df4000697c907', '5ea6ed2e080df4000697c908']}                                 3
{'reused': None, 'recovery_attempt': True, 'recovered': True, 'ships': ['5ea6ed2e080df4000697c908', '5ea6ed2e080df4000697c907']}                                 3
{'reused': Tr

In [19]:
#descomprimimos la columna de "fairings"
df[["reused", "recovery_attempt", "recovered"]] = df["fairings"].apply(pd.Series).iloc[:, :3]
df.head(1)

,fairings,links,static_fire_date_utc,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False


In [20]:
#ver columna
df["links"].value_counts()

links
{'patch': {'small': None, 'large': None}, 'reddit': {'campaign': None, 'launch': None, 'media': None, 'recovery': None}, 'flickr': {'small': [], 'original': []}, 'presskit': None, 'webcast': None, 'youtube_id': None, 'article': None, 'wikipedia': None}                                                                                                                                                                                                                                                                                                                                                                                                                15
{'patch': {'small': 'https://images2.imgbox.com/f9/4a/ZboXReNb_o.png', 'large': 'https://images2.imgbox.com/80/a2/bkWotCIS_o.png'}, 'reddit': {'campaign': None, 'launch': None, 'media': None, 'recovery': None}, 'flickr': {'small': [], 'original': []}, 'presskit': None, 'webcast': 'https://www.youtube.com/watch?v=Lk4zQ2wP-Nc', 'youtube_id

In [21]:
# descomprimir columna links, selección de 2 columnas mas relevantes
df[["webcast", "article"]] = df["links"].apply(pd.Series)[["webcast", "article"]]
df.head(1)

,fairings,links,static_fire_date_utc,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...


In [22]:
df["failures"].value_counts()

failures
[]                                                                                                                                                                                                                              200
[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]                                                                                                                                                               1
[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]                                                                                                                           1
[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]                                                                                                                 1
[{'time': 139, 'altitude': 40, 'reason': 'helium tank overpressure lead to the 

In [23]:
# descomprimir columna listas que tienen diccionarios
df[["time", "article", "reason"]] = df["failures"].apply(pd.Series)[0].apply(pd.Series).iloc[:, :3]
df.head(1)

,fairings,links,static_fire_date_utc,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,NaN,33.0,merlin engine failure


In [24]:
df.drop(["fairings", "links", "failures"], axis=1, inplace=True)

In [25]:
df.head(1)

,static_fire_date_utc,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason
0,2006-03-17T00:00:00.000Z,False,0.0,5e9d0d95eda69955f709d1eb,0.0,Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,NaN,33.0,merlin engine failure


In [26]:
# convertimos la columna de fecha a formato datetime
df["static_fire_date_utc"] = pd.to_datetime(df["static_fire_date_utc"])

In [27]:
df.sample(1)

,static_fire_date_utc,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason
10,2013-09-19 00:00:00+00:00,False,0.0,5e9d0d95eda69973a809d1ec,1.0,Commercial mission and first Falcon 9 v1.1 fli...,[],[5ea6ed2d080df4000697c903],[],[5eb0e4bbb6c3bb0006eeb1ee],5e9e4502f509092b78566f87,CASSIOPE,2013-09-29T16:00:00.000Z,1380470400,2013-09-29T09:00:00-07:00,hour,False,"[{'core': '5e9e289ff359180ae23b262d', 'flight'...",True,False,None,5eb87ce1ffd86e000604b334,False,False,False,https://www.youtube.com/watch?v=uFefasS6bhc,NaN,NaN,NaN


In [28]:
# creamos la columna de año y mes
df[['year', 'month']] = df['static_fire_date_utc'].apply(lambda x: pd.Series([x.year, x.month]))
df.head(2)

,static_fire_date_utc,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month
0,2006-03-17 00:00:00+00:00,False,0.0,5e9d0d95eda69955f709d1eb,0.0,Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,NaN,33.0,merlin engine failure,2006.0,3.0
1,NaT,False,0.0,5e9d0d95eda69955f709d1eb,0.0,Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b,False,False,False,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,289.0,301.0,harmonic oscillation leading to premature engi...,NaN,NaN


In [29]:
df.drop('static_fire_date_utc', axis=1, inplace=True)

In [30]:
df.sample(2)

,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month
168,False,NaN,5e9d0d95eda69973a809d1ec,1.0,None,[],[],[],[62adecbcd26f4f711fa53848],5e9e4501f509094ba4566f84,Globalstar FM15,2022-06-19T04:27:00.000Z,1655612820,2022-06-19T00:27:00-04:00,hour,False,"[{'core': '5f57c53d0622a6330279009f', 'flight'...",True,False,33223258-614c-449c-8af7-a9f75cc036b2,62a9f08b20413d2695d88711,None,None,None,https://youtu.be/94cClvOFWH4,NaN,NaN,NaN,NaN,NaN
44,False,0.0,5e9d0d95eda69973a809d1ec,1.0,"Dragon is expected to carry 2,349 kg (5,179 lb...",[],[5ea6ed30080df4000697c912],[5e9e2c5cf3591869b63b2670],[5eb0e4c4b6c3bb0006eeb212],5e9e4502f509094188566f88,CRS-12,2017-08-14T16:31:00.000Z,1502728260,2017-08-14T12:31:00-04:00,hour,False,"[{'core': '5e9e28a4f3591884ee3b264d', 'flight'...",True,False,None,5eb87d07ffd86e000604b356,NaN,NaN,NaN,https://www.youtube.com/watch?v=vLxWsYx8dbo,NaN,NaN,NaN,2017.0,8.0


In [31]:
# la columna 'success' esta con 0 y 1, lo cual es poco intuitivo. Vamos a sustituir los valores por "fracaso" y "exito"
mapa = {0: "Fracaso", 1: "Exito"}

# sobreescribimos el cambio en la columna original
df["success"] = df["success"].map(mapa)

# evaluamos que el cambio se realizó con exito
df["success"].unique()

array(['Fracaso', 'Exito', nan], dtype=object)

In [32]:
# tenemos la columna de 'window' que esta en segundos, vamos a convertirla a minutos. 
df["window_min"] = df["window"].apply(lambda x: x / 60)
df.head(1)

,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
0,False,0.0,5e9d0d95eda69955f709d1eb,Fracaso,Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,NaN,33.0,merlin engine failure,2006.0,3.0,0.0


In [33]:
df.drop(["window"], axis=1, inplace=True)

In [34]:
df.sample(3)

,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
124,False,5e9d0d95eda69973a809d1ec,Exito,This mission launches the 25th batch of operat...,[],"[608c1a06cf7f3d6152666ad4, 5ea6ed30080df400069...",[],[605b4befaa5433645e37d047],5e9e4502f509094188566f88,Starlink-25 (v1.0),2021-05-04T19:01:00.000Z,1620154860,2021-05-04T15:01:00-04:00,hour,False,"[{'core': '5e9e28a5f3591833b13b2659', 'flight'...",True,False,1ecc82c0-c5c8-41f0-aa58-b50a3b839ae0,605b4b7daa5433645e37d040,True,True,True,https://youtu.be/xpl_JnG7rcg,NaN,NaN,NaN,2021.0,5.0,0.0
170,False,5e9d0d95eda69973a809d1ec,Exito,None,[],[],[],[630bccc6d36448026ab01639],5e9e4501f509094ba4566f84,Starlink 4-21 (v1.5),2022-07-07T13:11:00.000Z,1657199460,2022-07-07T09:11:00-04:00,hour,False,"[{'core': '5e9e28a7f3591817f23b2663', 'flight'...",True,False,ac4ce8e1-fd76-4654-8809-5500ba792a8a,62a9f0c920413d2695d88712,None,None,None,https://youtu.be/u_A7xdnVllM,NaN,NaN,NaN,NaN,NaN,NaN
66,False,5e9d0d95eda69973a809d1ec,Exito,SpaceX's fifteenth flight of 2018 launched the...,[],"[5ea6ed2f080df4000697c90d, 5ea6ed30080df400069...",[],[5eb0e4c9b6c3bb0006eeb22a],5e9e4501f509094ba4566f84,Merah Putih,2018-08-07T05:18:00.000Z,1533619080,2018-08-07T01:18:00-04:00,hour,False,"[{'core': '5e9e28a5f359182b023b2656', 'flight'...",True,False,None,5eb87d20ffd86e000604b36c,False,False,False,https://www.youtube.com/watch?v=FjfQNBYv2IY,NaN,NaN,NaN,2018.0,8.0,120.0


#### Gestion de nulos

In [35]:
columnas_a_modificar = ['crew', 'ships', 'capsules', 'payloads']  # Agrega aquí todas las columnas que necesites

for columna in columnas_a_modificar:
    if columna in df.columns:
        df[columna] = df[columna].str.replace("[]", " ")

df.sample()

,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
22,False,5e9d0d95eda69973a809d1ec,Exito,None,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,TürkmenÄlem 52°E / MonacoSAT,2015-04-27T23:03:00.000Z,1430175780,2015-04-27T19:03:00-04:00,hour,False,"[{'core': '5e9e28a1f35918233f3b2639', 'flight'...",True,False,None,5eb87cedffd86e000604b340,False,False,False,https://www.youtube.com/watch?v=nBwAYT_ogj4,NaN,NaN,NaN,2015.0,4.0,0.0


In [36]:
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['success', 'details', 'launch_library_id', 'reused', 'recovery_attempt',
       'recovered', 'webcast', 'reason'],
      dtype='object')


In [37]:
# para todas las columnas tiene mas sentido reemplazar por desconocido
# iteramos por la lista de columnas a las que le vamos a cambiar los nulos por "Uknown"
for columna in nulos_esta_cat:
    
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df[columna] = df[columna].fillna("Unknown")
    
# comprobamos si quedan nulos en las columnas categóricas. 
print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df[nulos_esta_cat].isnull().sum()

Después del reemplazo usando 'fillna' quedan los siguientes nulos


success              0
details              0
launch_library_id    0
reused               0
recovery_attempt     0
recovered            0
webcast              0
reason               0
dtype: int64

In [38]:
nulos_esta_num = df[df.columns[df.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_esta_num)

Las columnas numéricas que tienen nulos son : 
 
Index(['crew', 'ships', 'capsules', 'payloads', 'article', 'time', 'year',
       'month', 'window_min'],
      dtype='object')


In [39]:
df_copia = df.copy()
df_copia.head(1)

,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
0,False,5e9d0d95eda69955f709d1eb,Fracaso,Engine failure at 33 seconds and loss of vehicle,NaN,NaN,NaN,NaN,5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,Unknown,5eb87cd9ffd86e000604b32a,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,NaN,33.0,merlin engine failure,2006.0,3.0,0.0


In [40]:
imputer_iterative = IterativeImputer(max_iter = 20, random_state = 42) #max_iter=20 especifica el número máximo de iteraciones/ random_state=42 asegura la reproducibilidad de los resultados.

# fit_transform() ajusta el modelo IterativeImputer a los datos seleccionados y luego transforma los datos imputando los valores faltantes.
imputer_iterative_imputado = imputer_iterative.fit_transform(df_copia[['crew', 'ships', 'capsules', 'payloads', 'article', 'time', 'year', 'month', 'window_min']])

# comprobamos que es lo que nos devuelve, que en este caso es un array también
imputer_iterative_imputado

C:\Users\InvitadoLaptop\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: [0 1 2 3]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


array([[ 1.15844765e+02,  3.30000000e+01,  2.00600000e+03,
         3.00000000e+00,  0.00000000e+00],
       [ 2.89000000e+02,  3.01000000e+02,  2.01745570e+03,
         6.65326444e+00,  0.00000000e+00],
       [ 3.50000000e+01,  1.40000000e+02,  2.01744277e+03,
         5.93401381e+00,  0.00000000e+00],
       ...,
       [ 9.14414054e+01, -4.04183208e+04,  2.01766982e+03,
         6.44424046e+00,  4.39713042e+01],
       [ 9.14414054e+01, -4.04183208e+04,  2.01766982e+03,
         6.44424046e+00,  4.39713042e+01],
       [ 9.14414054e+01, -4.04183208e+04,  2.01766982e+03,
         6.44424046e+00,  4.39713042e+01]], shape=(205, 5))

In [43]:
# Asigna los datos imputados al DataFrame original
df_copia[['article', 'time', 'year', 'month', 'window_min']] = imputer_iterative_imputado

# Comprueba los nulos
print(f"Después del 'Iterative' tenemos: \n{df_copia[['article', 'time', 'year', 'month', 'window_min']].isnull().sum()} nulos")

Después del 'Iterative' tenemos: 
article       0
time          0
year          0
month         0
window_min    0
dtype: int64 nulos


In [44]:
df.sample(3)

,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
60,False,5e9d0d95eda69973a809d1ec,Exito,First launch of a Block V first stage.,NaN,NaN,NaN,NaN,5e9e4502f509094188566f88,Bangabandhu-1,2018-05-11T20:14:00.000Z,1526069640,2018-05-11T16:14:00-04:00,hour,False,"[{'core': '5e9e28a5f359182b023b2656', 'flight'...",True,False,Unknown,5eb87d19ffd86e000604b366,False,False,False,https://www.youtube.com/watch?v=rQEqKZ7CJlk,NaN,NaN,Unknown,2018.0,5.0,127.0
64,False,5e9d0d95eda69973a809d1ec,Exito,SSL-manufactured communications satellite inte...,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Telstar 19V,2018-07-22T05:50:00.000Z,1532238600,2018-07-22T01:50:00-04:00,hour,False,"[{'core': '5e9e28a5f359181eed3b2657', 'flight'...",True,False,Unknown,5eb87d1effd86e000604b36a,False,False,False,https://www.youtube.com/watch?v=xybp6zLaGx4,NaN,NaN,Unknown,2018.0,7.0,120.0
54,False,5e9d0d95eda69974db09d1ed,Exito,"The launch was a success, and the side booster...",NaN,NaN,NaN,NaN,5e9e4502f509094188566f88,Falcon Heavy Test Flight,2018-02-06T20:45:00.000Z,1517949900,2018-02-06T15:45:00-05:00,hour,False,"[{'core': '5e9e28a5f359187f703b2653', 'flight'...",True,False,Unknown,5eb87d13ffd86e000604b360,False,False,False,https://www.youtube.com/watch?v=wbSwFU6tY1c,NaN,NaN,Unknown,2018.0,1.0,150.0


In [45]:
mover = df.pop('id')  # Eliminar y guardar la columna
df.insert(0, 'id', mover) 

In [46]:
df.sample(3)

,id,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
185,63161339ffc78f3b8567070c,False,5e9d0d95eda69973a809d1ec,Exito,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Starlink 4-35 (v1.5),2022-09-24T23:30:00.000Z,1664062200,2022-09-24T19:30:00-04:00,hour,False,"[{'core': '627843d657b51b752c5c5a53', 'flight'...",True,False,1c903b65-6667-4fd5-944d-296c5f13e01f,Unknown,Unknown,Unknown,https://youtu.be/VVu2bSJJhgI,NaN,NaN,Unknown,NaN,NaN,NaN
140,61bba806437241381bf7061e,False,5e9d0d95eda69973a809d1ec,Exito,The mission consists in launching 52 Starlink ...,NaN,NaN,NaN,NaN,5e9e4502f509092b78566f87,Starlink 4-4 (v1.5),2021-12-18T12:41:40.000Z,1639831300,2021-12-18T12:41:40-08:00,hour,False,"[{'core': '5e9e28a6f35918c0803b265c', 'flight'...",False,False,0d4b0c0f-3d72-4cb2-b596-dc526ad178a6,Unknown,True,Unknown,https://youtu.be/q4Ed3EBx90s,NaN,NaN,Unknown,2021.0,12.0,NaN
87,5eb87d3dffd86e000604b381,False,5e9d0d95eda69973a809d1ec,Exito,SpaceX will launch a Crew Dragon capsule from ...,NaN,NaN,NaN,NaN,5e9e4502f509094188566f88,Crew Dragon In Flight Abort Test,2020-01-19T14:00:00.000Z,1579442400,2020-01-19T09:00:00-05:00,hour,False,"[{'core': '5e9e28a5f359182b023b2656', 'flight'...",True,False,Unknown,Unknown,Unknown,Unknown,https://youtu.be/mhrkdHshb3E,NaN,NaN,Unknown,2020.0,1.0,240.0


### Filtrar datos

In [48]:
# ¿Qué cohetes se han lanzado entre el 2000 y el 2010? 
df[df["year"].between(2000, 2010)]

,id,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
0,5eb87cd9ffd86e000604b32a,False,5e9d0d95eda69955f709d1eb,Fracaso,Engine failure at 33 seconds and loss of vehicle,NaN,NaN,NaN,NaN,5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,Unknown,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,NaN,33.0,merlin engine failure,2006.0,3.0,0.0
3,5eb87cdbffd86e000604b32d,False,5e9d0d95eda69955f709d1eb,Exito,Ratsat was carried to orbit on the first succe...,NaN,NaN,NaN,NaN,5e9e4502f5090995de566f86,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",True,False,Unknown,False,False,False,https://www.youtube.com/watch?v=dLQ2tZEH6G0,NaN,NaN,Unknown,2008.0,9.0,0.0
5,5eb87cddffd86e000604b32f,False,5e9d0d95eda69973a809d1ec,Exito,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Falcon 9 Test Flight,2010-06-04T18:45:00.000Z,1275677100,2010-06-04T14:45:00-04:00,hour,False,"[{'core': '5e9e289ef359185f2b3b2628', 'flight'...",True,False,Unknown,Unknown,Unknown,Unknown,https://www.youtube.com/watch?v=nxSxgBKlYws,NaN,NaN,Unknown,2010.0,3.0,0.0
6,5eb87cdeffd86e000604b330,False,5e9d0d95eda69973a809d1ec,Exito,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,COTS 1,2010-12-08T15:43:00.000Z,1291822980,2010-12-08T11:43:00-04:00,hour,False,"[{'core': '5e9e289ef35918187c3b2629', 'flight'...",True,False,Unknown,Unknown,Unknown,Unknown,https://www.youtube.com/watch?v=cdLITgWKe_0,NaN,NaN,Unknown,2010.0,12.0,0.0


In [49]:
# seleccionamos aquellas filas donde 'date_precision' sea igual a day
df[df["date_precision"] == "day"]

,id,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
175,62a9f12820413d2695d88716,False,5e9d0d95eda69973a809d1ec,Exito,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Starlink 4-25 (v1.5),2022-07-24T00:00:00.000Z,1658620800,2022-07-23T20:00:00-04:00,day,False,"[{'core': '5f57c5440622a633027900a0', 'flight'...",True,False,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,Unknown,NaN,NaN,NaN
192,633f71a90531f07b4fdf59be,False,5e9d0d95eda69973a809d1ec,Unknown,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Galaxy 31 (23R) & 32 (17R),2022-11-08T00:00:00.000Z,1667865600,2022-11-07T19:00:00-05:00,day,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,Unknown,NaN,NaN,NaN
193,633f71b60531f07b4fdf59bf,False,5e9d0d95eda69973a809d1ec,Unknown,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Eutelsat 10B,2022-11-15T00:00:00.000Z,1668470400,2022-11-14T19:00:00-05:00,day,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,Unknown,NaN,NaN,NaN
194,633f723d0531f07b4fdf59c4,False,5e9d0d95eda69973a809d1ec,Unknown,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,ispace Mission 1 & Rashid,2022-11-22T00:00:00.000Z,1669075200,2022-11-21T19:00:00-05:00,day,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,Unknown,NaN,NaN,NaN
198,633f724c0531f07b4fdf59c5,False,5e9d0d95eda69973a809d1ec,Unknown,Unknown,NaN,NaN,NaN,NaN,5e9e4502f509092b78566f87,SWOT,2022-12-05T00:00:00.000Z,1670198400,2022-12-04T16:00:00-08:00,day,True,"[{'core': None, 'flight': None, 'gridfins': Tr...",True,False,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,Unknown,NaN,NaN,NaN


In [50]:
# al resultado anterior le añadimos la condición de que sea el lanzamiento haya ocurrido con exito
df[(df["date_precision"] == "day") & (df["success"] == "Exito")]

,id,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
175,62a9f12820413d2695d88716,False,5e9d0d95eda69973a809d1ec,Exito,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Starlink 4-25 (v1.5),2022-07-24T00:00:00.000Z,1658620800,2022-07-23T20:00:00-04:00,day,False,"[{'core': '5f57c5440622a633027900a0', 'flight'...",True,False,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,Unknown,NaN,NaN,NaN


In [51]:
# que cohetes tienen en el nombre "Falcon". 

df[df["name"].str.contains("Falcon")]

,id,net,rocket,success,details,crew,ships,capsules,payloads,launchpad,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,reused,recovery_attempt,recovered,webcast,article,time,reason,year,month,window_min
0,5eb87cd9ffd86e000604b32a,False,5e9d0d95eda69955f709d1eb,Fracaso,Engine failure at 33 seconds and loss of vehicle,NaN,NaN,NaN,NaN,5e9e4502f5090995de566f86,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,Unknown,False,False,False,https://www.youtube.com/watch?v=0a_00nJ_Y88,NaN,33.0,merlin engine failure,2006.0,3.0,0.0
5,5eb87cddffd86e000604b32f,False,5e9d0d95eda69973a809d1ec,Exito,Unknown,NaN,NaN,NaN,NaN,5e9e4501f509094ba4566f84,Falcon 9 Test Flight,2010-06-04T18:45:00.000Z,1275677100,2010-06-04T14:45:00-04:00,hour,False,"[{'core': '5e9e289ef359185f2b3b2628', 'flight'...",True,False,Unknown,Unknown,Unknown,Unknown,https://www.youtube.com/watch?v=nxSxgBKlYws,NaN,NaN,Unknown,2010.0,3.0,0.0
54,5eb87d13ffd86e000604b360,False,5e9d0d95eda69974db09d1ed,Exito,"The launch was a success, and the side booster...",NaN,NaN,NaN,NaN,5e9e4502f509094188566f88,Falcon Heavy Test Flight,2018-02-06T20:45:00.000Z,1517949900,2018-02-06T15:45:00-05:00,hour,False,"[{'core': '5e9e28a5f359187f703b2653', 'flight'...",True,False,Unknown,False,False,False,https://www.youtube.com/watch?v=wbSwFU6tY1c,NaN,NaN,Unknown,2018.0,1.0,150.0


In [52]:
# ¿Qué año se lanzaron mas cohetes? 
df.groupby(['year'])["id"].count().reset_index()

,year,id
0,2006.0,1
1,2008.0,1
2,2010.0,2
3,2012.0,2
4,2013.0,4
5,2014.0,5
6,2015.0,7
7,2016.0,9
8,2017.0,19
9,2018.0,20


In [53]:
# ¿Qué  mes se lanzaron mas cohetes? 
df.groupby(['month'])["id"].count().reset_index()

,month,id
0,1.0,11
1,2.0,9
2,3.0,10
3,4.0,12
4,5.0,11
5,6.0,13
6,7.0,6
7,8.0,9
8,9.0,9
9,10.0,8


In [54]:
# ¿Cuántos lanzamientos hubo con éxito? 
df.groupby("success")["id"].count().reset_index()

,success,id
0,Exito,181
1,Fracaso,5
2,Unknown,19


In [55]:
# hacemos lo mismo pero calculandolo en %
df_exito = df.groupby("success")["id"].count().reset_index()
df_exito['porcentaje'] = (df_exito['id'] / df_exito['id'].sum()) * 100
df_exito

,success,id,porcentaje
0,Exito,181,88.292683
1,Fracaso,5,2.439024
2,Unknown,19,9.268293


In [56]:
# ¿Cuál es la ventana de duración media, mínima y máxima por año?
df.groupby("year")["window_min"].agg(["mean", "min", "max", "count"]).round(2)

,mean,min,max,count
year,,,,
2006.0,0.00,0.0,0.0,1
2008.0,0.00,0.0,0.0,1
2010.0,0.00,0.0,0.0,2
2012.0,0.00,0.0,0.0,2
2013.0,0.00,0.0,0.0,4
2014.0,24.00,0.0,120.0,5
2015.0,0.00,0.0,0.0,7
2016.0,61.88,0.0,120.0,8
2017.0,72.53,0.0,305.0,19
